In [1]:
!pip install -qqq tiktoken
!pip install -qqq pinecone-client
!pip install -qqq openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [2]:

!pip install -qU langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.9 MB/s eta 0:00:00


In [3]:


from google.colab import userdata
OPENAI_API_KEY=userdata.get('openai-key')
PINECONE_API_KEY=userdata.get('pinecone-key')




In [4]:
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

##Load the CSV Data Using Pandas

In [9]:
import pandas as pd

# Load the CSV file (adjust the path as needed)
data = pd.read_csv('/content/card_related_dataset.csv')
# Display the first few rows to verify the data
print(data.head())


                                         instruction         intent  \
0   I would like to acivate a card, can you help me?  activate_card   
1  I have to activate an Visa online, how can I d...  activate_card   
2        I'd like to actiate a card where do i do it  activate_card   
3  I'd likke to activate a visa on mobile i need ...  activate_card   
4  I would ilke to activate a credit card online,...  activate_card   

                                            response  
0  I'm here to assist you with that! Activating y...  
1  I'm here to assist you with activating your {{...  
2  I can help you with that! Activating your card...  
3  I'm here to assist you with activating your {{...  
4  I'm here to assist you with activating your cr...  


##Generating Embeddings with langchain_openai

In [11]:
from langchain_openai import OpenAIEmbeddings
import time

# Initialize the embedding model
embed_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Function to generate embeddings in batches
def generate_embeddings_in_batches(documents, batch_size=100):
    embeddings = []
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i+batch_size]
        try:
            # Generate embeddings for the batch and extend to the main list
            batch_embeddings = embed_model.embed_documents(batch)
            embeddings.extend(batch_embeddings)
            print(f"Processed batch {i//batch_size + 1}/{len(documents)//batch_size + 1}")
        except Exception as e:
            print(f"Error in batch {i//batch_size + 1}: {e}")
            time.sleep(5)  # Wait a bit before retrying in case of API rate limits
    return embeddings

# Generate embeddings for each instruction in the CSV in batches
data['instruction_embedding'] = generate_embeddings_in_batches(data['instruction'].tolist())

# Verify the dimension of the embeddings
print(f"Embedding dimension: {len(data['instruction_embedding'][0])}")
print(f"Number of embeddings generated: {len(data['instruction_embedding'])}")


Processed batch 1/256
Processed batch 2/256
Processed batch 3/256
Processed batch 4/256
Processed batch 5/256
Processed batch 6/256
Processed batch 7/256
Processed batch 8/256
Processed batch 9/256
Processed batch 10/256
Processed batch 11/256
Processed batch 12/256
Processed batch 13/256
Processed batch 14/256
Processed batch 15/256
Processed batch 16/256
Processed batch 17/256
Processed batch 18/256
Processed batch 19/256
Processed batch 20/256
Processed batch 21/256
Processed batch 22/256
Processed batch 23/256
Processed batch 24/256
Processed batch 25/256
Processed batch 26/256
Processed batch 27/256
Processed batch 28/256
Processed batch 29/256
Processed batch 30/256
Processed batch 31/256
Processed batch 32/256
Processed batch 33/256
Processed batch 34/256
Processed batch 35/256
Processed batch 36/256
Processed batch 37/256
Processed batch 38/256
Processed batch 39/256
Processed batch 40/256
Processed batch 41/256
Processed batch 42/256
Processed batch 43/256
Processed batch 44/2

##Create Unique IDs and Combine Metadata

In [12]:
# Create unique IDs for each instruction
ids = [str(i) for i in range(len(data))]

# Function to combine embeddings with metadata for Pinecone upload
def prepare_data_for_pinecone(documents, doc_embeddings):
    data_with_metadata = []
    for doc_id, doc_text, embedding in zip(ids, documents, doc_embeddings):
        # Ensure text is a string
        doc_text = str(doc_text) if not isinstance(doc_text, str) else doc_text

        # Create data item with ID, embedding, and metadata
        data_item = {
            "id": doc_id,
            "values": embedding,
            "metadata": {"text": doc_text},
        }
        data_with_metadata.append(data_item)
    return data_with_metadata

# Prepare the data for Pinecone upload
all_meta_data = prepare_data_for_pinecone(data['instruction'].tolist(), data['instruction_embedding'].tolist())


##Initialize Pinecone and Create the Index

In [13]:
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone
pinecone = Pinecone()

INDEX_NAME = "credit-card-instructions"

# Check if the index already exists and delete it if needed
if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
    pinecone.delete_index(INDEX_NAME)

# Create the index with specified dimensions and cosine similarity metric
pinecone.create_index(
    name=INDEX_NAME,
    dimension=len(data['instruction_embedding'][0]),  # Adjust dimension based on embedding size
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

# Connect to the created index
index = pinecone.Index(INDEX_NAME)


##Upsert Embeddings and Metadata into Pinecone in Batches

In [15]:
# Function to upsert data to Pinecone in batches
def upsert_in_batches(data, batch_size=100):
    for i in range(0, len(data), batch_size):
        batch = data[i:i + batch_size]
        index.upsert(vectors=batch)
        print(f"Uploaded batch {i // batch_size + 1} of {len(data) // batch_size + 1}")

# Upsert embeddings and metadata in batches
upsert_in_batches(all_meta_data)
print("All data successfully uploaded to Pinecone in batches.")



Uploaded batch 1 of 256
Uploaded batch 2 of 256
Uploaded batch 3 of 256
Uploaded batch 4 of 256
Uploaded batch 5 of 256
Uploaded batch 6 of 256
Uploaded batch 7 of 256
Uploaded batch 8 of 256
Uploaded batch 9 of 256
Uploaded batch 10 of 256
Uploaded batch 11 of 256
Uploaded batch 12 of 256
Uploaded batch 13 of 256
Uploaded batch 14 of 256
Uploaded batch 15 of 256
Uploaded batch 16 of 256
Uploaded batch 17 of 256
Uploaded batch 18 of 256
Uploaded batch 19 of 256
Uploaded batch 20 of 256
Uploaded batch 21 of 256
Uploaded batch 22 of 256
Uploaded batch 23 of 256
Uploaded batch 24 of 256
Uploaded batch 25 of 256
Uploaded batch 26 of 256
Uploaded batch 27 of 256
Uploaded batch 28 of 256
Uploaded batch 29 of 256
Uploaded batch 30 of 256
Uploaded batch 31 of 256
Uploaded batch 32 of 256
Uploaded batch 33 of 256
Uploaded batch 34 of 256
Uploaded batch 35 of 256
Uploaded batch 36 of 256
Uploaded batch 37 of 256
Uploaded batch 38 of 256
Uploaded batch 39 of 256
Uploaded batch 40 of 256
Uploaded 

##Generate Query Embedding and Search Pinecone

In [16]:
def query_pinecone_for_relevant_docs(query, top_k=3):
    """
    Embed the user query and retrieve relevant documents from Pinecone.

    Args:
    - query (str): The user’s question.
    - top_k (int): Number of top matches to retrieve from Pinecone.

    Returns:
    - list: A list of tuples containing similarity scores and document texts.
    """
    # Generate the query embedding
    query_embedding = embed_model.embed_query(query)

    # Query Pinecone index with the query embedding
    query_response = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    # Extract similarities and document texts
    results = [(match['score'], match['metadata']['text']) for match in query_response['matches']]
    return results


##Filter Results for Relevance

In [17]:
def filter_relevance(results, threshold=0.8):
    """
    Filter the results based on a similarity threshold to ensure relevance.

    Args:
    - results (list): A list of tuples with similarity scores and document texts.
    - threshold (float): Minimum similarity score to be considered relevant.

    Returns:
    - str: Relevant document text or a message indicating off-topic content.
    """
    # Check if any result meets the relevance threshold
    for score, document in results:
        if score >= threshold:
            return document

    # Return a message if no document meets the threshold
    return "The question is beyond the scope of the current documents."


##Use LLM for Final Response

In [18]:
from langchain_openai import ChatOpenAI

# Initialize the LLM
LLM = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

def generate_llm_response(retrieved_document, query):
    """
    Generate a final response using the LLM.

    Args:
    - retrieved_document (str): The relevant document retrieved from Pinecone.
    - query (str): The original user query.

    Returns:
    - str: The final response from the LLM.
    """
    prompt = f"{retrieved_document} Using the provided information, answer the following question: {query}"
    answer = LLM.invoke(prompt)
    return answer.content


##To streamline the workflow, let’s add the main function, handle_user_query, which will tie everything together.

In [34]:
def handle_user_query(query, threshold=0.8, top_k=3):
    """
    Process a user query by embedding it, retrieving relevant documents, and generating a response.

    Args:
    - query (str): User’s question.
    - threshold (float): Minimum similarity score for relevance.
    - top_k (int): Number of top results to retrieve from Pinecone.

    Returns:
    - str: The final answer from the LLM or an off-topic message.
    """
    # Step 1: Retrieve relevant documents from Pinecone
    retrieval_results = query_pinecone_for_relevant_docs(query, top_k=top_k)

    # Step 2: Filter results based on relevance threshold
    relevant_document = filter_relevance(retrieval_results, threshold=threshold)

    # Step 3: Generate response if relevant; otherwise, return off-topic message
    if relevant_document == "The question is beyond the scope of the current documents.":
        return relevant_document
    else:
        return generate_llm_response(relevant_document, query)


##Query1 (Out of scope for our chatbot)
"Can you teach me how to drive a truck?"

In [38]:
# Example query
query1 = "Can you teach me how to drive a truck?"

# Step 1: Get the response from the red teaming function to check relevance
response = handle_user_query(query1)  # This will handle retrieval and relevance filtering

# Initialize the LLM
from langchain_openai import ChatOpenAI
LLM = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Step 2: Create the LLM prompt based on the response
if response == "The question is beyond the scope of the current documents.":
    print("I'm sorry, but I don't have information on that topic. Please try asking about something related to credit card services.")
else:
    # Construct prompt using the relevant document and original query
    prompt = f"{response} Using the provided information, answer the following question: {query1}"

    # Step 3: Generate the final answer with LLM
    def query_response(prompt):
        answer = LLM.invoke(prompt)
        return answer.content

    # Get and print the final answer
    final_answer = query_response(prompt=prompt)
    print(f"\nResponse from the LLM:\n{final_answer}")


I'm sorry, but I don't have information on that topic. Please try asking about something related to credit card services.


##Query2
Can you help me in activating the credit card?

In [37]:
# Example query
query2 = "Can you help me in activating the credit card?"

# Step 1: Get the response from the red teaming function to check relevance
response = handle_user_query(query2)  # This will handle retrieval and relevance filtering

# Initialize the LLM
from langchain_openai import ChatOpenAI
LLM = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Step 2: Create the LLM prompt based on the response
if response == "The question is beyond the scope of the current documents.":
    print("I'm sorry, but I don't have information on that topic. Please try asking about something related to credit card services.")
else:
    # Construct prompt using the relevant document and original query
    prompt = f"{response} Using the provided information, answer the following question: {query2}"

    # Step 3: Generate the final answer with LLM
    def query_response(prompt):
        answer = LLM.invoke(prompt)
        return answer.content

    # Get and print the final answer
    final_answer = query_response(prompt=prompt)
    print(f"\nResponse from the LLM:\n{final_answer}")


Response from the LLM:
Yes, I can help you activate your credit card. Please provide me with the necessary information and I will guide you through the activation process.


##Query3

I'd like information about my recent transactions were can i find it

In [42]:
# Example query
query3 = "I'd like information about my recent transactions were can i find it"

# Step 1: Get the response from the red teaming function to check relevance
response = handle_user_query(query3)  # This will handle retrieval and relevance filtering

# Initialize the LLM
from langchain_openai import ChatOpenAI
LLM = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Step 2: Create the LLM prompt based on the response
if response == "The question is beyond the scope of the current documents.":
    print("I'm sorry, but I don't have information on that topic. Please try asking about something related to credit card services.")
else:
    # Construct prompt using the relevant document and original query
    prompt = f"{response} Using the provided information, answer the following question: {query3}"

    # Step 3: Generate the final answer with LLM
    def query_response(prompt):
        answer = LLM.invoke(prompt)
        return answer.content

    # Get and print the final answer
    final_answer = query_response(prompt=prompt)
    print(f"\nResponse from the LLM:\n{final_answer}")


Response from the LLM:
You can find information about your recent transactions by checking your bank statement online or through your mobile banking app. You can also contact your bank directly for assistance in accessing this information.


##Query 4
I need to recover a Visa by an ATM, where do I do it?

In [41]:
# Example query
query4 = "How to pay the card bill?"

# Step 1: Get the response from the red teaming function to check relevance
response = handle_user_query(query4)  # This will handle retrieval and relevance filtering

# Initialize the LLM
from langchain_openai import ChatOpenAI
LLM = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Step 2: Create the LLM prompt based on the response
if response == "The question is beyond the scope of the current documents.":
    print("I'm sorry, but I don't have information on that topic. Please try asking about something related to credit card services.")
else:
    # Construct prompt using the relevant document and original query
    prompt = f"{response} Using the provided information, answer the following question: {query4}"

    # Step 3: Generate the final answer with LLM
    def query_response(prompt):
        answer = LLM.invoke(prompt)
        return answer.content

    # Get and print the final answer
    final_answer = query_response(prompt=prompt)
    print(f"\nResponse from the LLM:\n{final_answer}")


Response from the LLM:
You can pay the card bill by making a payment online through the credit card issuer's website, setting up automatic payments, paying by phone, mailing a check, or visiting a branch of the issuing bank.


##Query6
Who won the fifa worldcup last year?

In [40]:
# Example query
query6 = "Who won the fifa worldcup last year?"

# Step 1: Get the response from the red teaming function to check relevance
response = handle_user_query(query6)  # This will handle retrieval and relevance filtering

# Initialize the LLM
from langchain_openai import ChatOpenAI
LLM = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Step 2: Create the LLM prompt based on the response
if response == "The question is beyond the scope of the current documents.":
    print("I'm sorry, but I don't have information on that topic. Please try asking about something related to credit card services.")
else:
    # Construct prompt using the relevant document and original query
    prompt = f"{response} Using the provided information, answer the following question: {query6}"

    # Step 3: Generate the final answer with LLM
    def query_response(prompt):
        answer = LLM.invoke(prompt)
        return answer.content

    # Get and print the final answer
    final_answer = query_response(prompt=prompt)
    print(f"\nResponse from the LLM:\n{final_answer}")

I'm sorry, but I don't have information on that topic. Please try asking about something related to credit card services.
